In [2]:
import pandas as pd
base = pd.read_csv('credit_data.csv')
base.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [3]:
base.shape

(2000, 5)

In [4]:
base = base.drop('i#clientid', axis = 1)
base.head()

,income,age,loan,c#default
0,66155.925095,59.017015,8106.532131,0
1,34415.153966,48.117153,6564.745018,0
2,57317.170063,63.108049,8020.953296,0
3,42709.534201,45.751972,6103.642260,0
4,66952.688845,18.584336,8770.099235,1


In [5]:
base = base.dropna()
base.shape

(1997, 4)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
base.head()

,income,age,loan,c#default
0,1.453898,1.336861,1.201907,0
1,-0.762398,0.536639,0.695744,0
2,0.836733,1.637207,1.173812,0
3,-0.183244,0.362998,0.544366,0
4,1.509532,-1.631534,1.419754,1


In [7]:
X = base.drop('c#default', axis = 1)
y = base['c#default']

In [8]:
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [9]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: c#default, dtype: int64

In [15]:
X.columns

Index(['income', 'age', 'loan'], dtype='object')

In [11]:
import tensorflow as tf
tf.__version__

2022-06-27 19:38:12.514415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-27 19:38:12.514468: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.9.0'

In [12]:
colunas = [tf.feature_column.numeric_column(key = column) for column in X.columns]

In [14]:
colunas

[NumericColumn(key='income', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [13]:
print(colunas[2])

NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [13]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [14]:
X_treinamento.shape

(1397, 3)

In [15]:
X_teste.shape

(600, 3)

In [16]:
def make_input_fn(X, y = None, num_epochs = None, shuffle = True, batch_size = 8):
    def input_function():
        # As variáveis previsoras precisam ser passadas na função seguinte como um dicionário com os mesmos nomes definidos na chamada de `tf.feature_column.numeric_column`
        ds = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [17]:
funcao_treinamento = make_input_fn(X_treinamento, y_treinamento)

In [18]:
config = tf.estimator.RunConfig(tf_random_seed = 42)
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4], config = config)
classificador.train(input_fn = funcao_treinamento, steps = 1000)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqkndwspd', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:t

In [19]:
funcao_teste = make_input_fn(X_teste, y_teste, num_epochs = 1000, shuffle = False)

In [20]:
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-08T18:35:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqkndwspd/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [

In [21]:
metricas_teste

{'accuracy': 0.7105,
 'accuracy_baseline': 0.848875,
 'auc': 0.27555847,
 'auc_precision_recall': 0.09800053,
 'average_loss': 0.64567107,
 'global_step': 1000,
 'label/mean': 0.151125,
 'loss': 0.64567107,
 'precision': 0.0,
 'prediction/mean': 0.44303054,
 'recall': 0.0}